# Perform lineage tracking

In [177]:
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy

import skimage
from skimage import measure, transform # to get contours from masks

# btrack module and configuration file
import btrack
from btrack.dataio import localizations_to_objects
from btrack.constants import BayesianUpdates
from btrack.render import plot_tracks

import imagecodecs
import napari

# 1. Load segmentation and tracking results for {Whole Cell, Bud}

In [82]:
os.chdir("D:\Hugo\Pantrack\BF_f0001")  # root dir containg Predictions and Images Path

wc_prediction_path = "Sd100_BF_f0001.tif"
wc_tracking_path = "BF_f0001_whole_cell_tracking.csv"
bud_prediction_path = "B165_BF_f0001.tif"
bud_tracking_path = "BF_f0001_bud_tracking.csv"
corresponding_imgs = "BF_f0001.tif"

wc_masks = imageio.volread(wc_prediction_path)
wc_tracking = pd.read_csv(wc_tracking_path)
wc_contours = np.load("BF_f0001_whole_cell_contours.npy", allow_pickle=True)
bud_masks = imageio.volread(bud_prediction_path)
bud_tracking = pd.read_csv(bud_tracking_path)
bud_contours = np.load("BF_f0001_bud_contours.npy", allow_pickle=True)

wc_tracking["Contours"] = wc_contours
bud_tracking["Contours"] = bud_contours

wc_tracking["Parent"] = wc_tracking["ID"]  # default parent value is ID.

imgs = imageio.volread(corresponding_imgs)
if imgs.shape[1] != wc_masks.shape[1] or imgs.shape[2] != wc_masks.shape[2]:
    imgs = skimage.transform.resize(imgs, (imgs.shape[0], wc_masks.shape[1], wc_masks.shape[2]))
if bud_masks.shape[1] != wc_masks.shape[1] or bud_masks.shape[2] != wc_masks.shape[2]:
    bud_masks = skimage.transform.resize(bud_masks, (bud_masks.shape[0], wc_masks.shape[1], wc_masks.shape[2]))
    
print(wc_masks.shape, bud_masks.shape, imgs.shape)

# Cast to 8-bit depth images prior to thresholding
wc_masks_8b = ((wc_masks - wc_masks.min(axis=(0, 1))) / (wc_masks.max(axis=(0, 1)) - wc_masks.min(axis=(0, 1)))).astype("uint8")
bud_masks_8b = ((bud_masks - bud_masks.min(axis=(0, 1))) / (bud_masks.max(axis=(0, 1)) - bud_masks.min(axis=(0, 1)))).astype("uint8")

(160, 512, 512) (160, 512, 512) (160, 512, 512)


In [183]:
def visualize_data_and_predictions(bf, predictions1, predictions2, nc_ims=1, nc_masks=1):
    
    if nc_ims == 1:
        viewer = napari.view_image(bf[:, :, :])
    else:
        viewer = napari.view_image(bf[:, :, :, 0])  # bf
        for k in range(1, nc_ims):
            viewer.add_image(bf[:, :, :, k], blending="additive")
    
    if predictions1 is not None:
        pred1, track1 = predictions1
        if nc_masks == 1:
            viewer.add_image(pred1[:, :, :], blending="additive", colormap="blue")
        else:
            cmaps = ["bop blue", "red", "bop_orange", "blue", "bop purple"]
            for k in range(0, nc_masks):
                viewer.add_image(pred1[:, :, :, k], blending="additive", colormap=cmaps[k])
        print(track1.columns)
        viewer.add_tracks(track1.drop(["Contours", "Parent"], axis=1))
    
    if predictions2 is not None:
        pred2, track2 = predictions2
        if nc_masks == 1:
            viewer.add_image(pred2[:, :, :], blending="additive", colormap="bop purple")
        else:
            cmaps = ["bop blus", "red", "bop_orange", "blue", "bop purple"]
            for k in range(0, nc_masks):
                viewer.add_image(pred2[:, :, :, k], blending="additive", colormap=cmaps[len(cmaps) - 1 - k])
        viewer.add_tracks(track2.drop(["Contours"], axis=1))
    

visualize_data_and_predictions(imgs, (wc_masks, wc_tracking), (bud_masks, bud_tracking), nc_ims=1, nc_masks=1)
wc_tracking.head()

Index(['ID', 'Frame', 'X', 'Y', 'Contours', 'Parent'], dtype='object')


,ID,Frame,X,Y,Contours,Parent
0,1,0,191.445904,319.614458,"[[201.01, 322.0], [201.01, 321.0], [201.01, 32...",1
1,1,1,191.223647,319.835294,"[[201.01, 321.0], [201.01, 320.0], [201.01, 31...",1
2,1,2,188.605062,320.320988,"[[198.01, 321.0], [198.01, 320.0], [198.01, 31...",1
3,1,3,189.060361,320.481928,"[[199.01, 320.0], [199.01, 319.0], [199.0, 318...",1
4,1,4,190.060361,320.481928,"[[200.01, 320.0], [200.0, 319.99], [199.01, 31...",1


In [180]:
wc_tracking = wc_tracking.drop("Root", axis=1)

# 2. Link the tracking results between both modules

For each budding event (considered as a true positive), we want to retrieve the two maximum matching whole cell segmentation : they are the mother and daughter cell.

In [182]:
import scipy

def distance_criterion(bud_object, whole_cell_objects):
    """
    To find the best match between whole cell objects and bud objects, computes the distance
    between the centroid of the bud object and all the whole cell objects at the same frame
    and takes the two closest whole cell objects.
    bud_object (pd.Series): bud point at one frame
    whole_cell_objects (pd.DataFrame): all the whole cell objects in a dataframe
    Return: the two best match indices
    """
    def euclidean_distance(x1, y1, x2, y2):
        return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    
    x, y, frame = bud_object["X"], bud_object["Y"], bud_object["Frame"]
    
    # get the whole cell tracklets at this frame and retrieve the two maximum matching points (e.g. the two closest)
    at_this_frame = whole_cell_objects[whole_cell_objects["Frame"] == frame]
    barycenters = at_this_frame[["X", "Y"]]
    distances = euclidean_distance(x, y, barycenters.values[:, 0], barycenters.values[:, 1])
    sorted_distances = np.sort(distances)
    
    if sorted_distances[0] > 20 or sorted_distances[1] > 20:
        return None
    
    min_idx, min_idx1 = np.argwhere(distances == sorted_distances[0])[0, 0], np.argwhere(distances == sorted_distances[1])[0, 0]
    
    closest_indices = at_this_frame.iloc[[min_idx, min_idx1]]["ID"].values # mom and daughter index
    
    return closest_indices

def intersection_criterion(bud_object, whole_cell_objects):
    """
    To find the best match between whole cell objects and bud objects, computes the intersection between
    the bud object and all the whole cell objects at the same frame and takes the largest overlaps.
    bud_object (pd.Series): bud point at one frame
    whole_cell_objects (pd.DataFrame): all the whole cell objects in a dataframe
    Return: the two best match indices
    """
    # get the whole cell tracklets at this frame and retrieve the two maximum matching points (e.g. the two closest)
    at_this_frame = whole_cell_objects[whole_cell_objects["Frame"] == frame]
    
    x, y = bud_object["X"], bud_object["Y"]
    
    def euclidean_distance(x1, y1, x2, y2):
        return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    
    def get_mask(series):
        mask = np.zeros((512, 512))
        if np.isnan(series["Contours"]).any():
            return mask
        if euclidean_distance(x, y, series["X"], series["Y"]) > 20:  # filter the candidates, avoid useles computing -> speed up a lot
            return mask
        contours = series["Contours"].astype("uint8")
        mask[contours[:, 1], contours[:, 0]] = 1
        masked_image = scipy.ndimage.morphology.binary_fill_holes(mask)
        return masked_image
    
    # make masks array from contours for bud object
    bud_mask = get_mask(bud_object)

    # make masks for the whole cell objects also
    wc_masks = np.array([get_mask(row) for i, row in at_this_frame.iterrows()])
    
    # compute intersections
    intersections = np.array([np.sum(np.logical_and(bud_mask, m)) for m in wc_masks])
    
    intersection_indices = np.argwhere(intersections != 0).squeeze(-1)
    
    if intersection_indices.size < 2:
        return None
    
    if (intersection_indices < 5).any():
        return None
    
    return at_this_frame.iloc[intersection_indices]["ID"].values


# distance or intersection
import time
criterion = "intersection"
t0 = time.time()

retrieved_mothers = 0
for i, bud_traj_id in enumerate(bud_tracking["ID"].unique()):
    bud_traj = bud_tracking[bud_tracking["ID"] == bud_traj_id].iloc[0]
    
    frame = bud_traj["Frame"]
    
    if bud_traj_id % 50 == 0:
        print(f"Bud ID : {bud_traj_id}")
    
    # distance criterion
    if criterion == "distance":
        closest_indices = distance_criterion(bud_traj, wc_tracking)
    elif criterion == "intersection":
        closest_indices = intersection_criterion(bud_traj, wc_tracking)
        
    if closest_indices is None:
        continue
    
    # now determine who is the mother and who is the daughter. # criterion: mother should be bigger and older
    candidates = [wc_tracking[wc_tracking["ID"] == closest_indices[0]], wc_tracking[wc_tracking["ID"] == closest_indices[1]]]
    
    # no contours at this frame
    if np.isnan(candidates[0][candidates[0]["Frame"] == frame]["Contours"].values[0]).any() or np.isnan(candidates[1][candidates[1]["Frame"] == frame]["Contours"].values[0]).any():
        continue
        
    sizes = [cv2.contourArea(candidates[0][candidates[0]["Frame"] == frame]["Contours"].values[0].astype("float32")), cv2.contourArea(candidates[1][candidates[1]["Frame"] == frame]["Contours"].values[0].astype("float32"))]
    first_frames = [np.min(candidates[0]["Frame"].values), np.min(candidates[1]["Frame"].values)]
    older_idx, bigger_idx = np.argmin(first_frames), np.argmax(sizes)
    
    if older_idx == bigger_idx:  # agreement between the two criteria
        mother_idx = older_idx
        daughter_idx = 1 - older_idx
        wc_tracking.loc[wc_tracking["ID"] == closest_indices[daughter_idx], "Parent"] = closest_indices[mother_idx]
        
        print(f"Frame {frame}, Track {closest_indices[mother_idx]} mother of track {closest_indices[daughter_idx]}")

        retrieved_mothers += 1
    else:
        continue  # mother and daughter could not be determined
        
    
    
n_trajectories_total = wc_tracking["ID"].unique().shape[0] - wc_tracking[wc_tracking["Frame"] == 0]["ID"].unique().shape[0]
print(f"Retrieved {retrieved_mothers} lineages over {n_trajectories_total}.")
print(f"Duration: {round(time.time() - t0, 2)}.")

Frame 0, Track 13 mother of track 17
Frame 3, Track 12 mother of track 20
Frame 3, Track 25 mother of track 32
Frame 3, Track 7 mother of track 42
Frame 4, Track 27 mother of track 29
Frame 7, Track 13 mother of track 34
Frame 12, Track 13 mother of track 52
Frame 16, Track 13 mother of track 17
Frame 20, Track 16 mother of track 68


KeyboardInterrupt: 

In [ ]:
wc_tracking.drop("Contours", axis=1).to_csv("D:\Hugo\Pantrack\BF_f0001/BF_f0001_lineage_tracking.csv", sep=",")